In [1]:
import joblib
import os
from EXE_scanner import EXEscanner

In [2]:
class WrapperEXEscanner:
    def __init__(self, main_classifier, EXE_scanner, main_classifier_threshold=None):
        self.main_classifier = main_classifier
        self.EXE_scanner = EXE_scanner
        self.main_classifier_threshold = main_classifier_threshold

    def predict_sample(self, bytez, return_score=False):
        if return_score:
            assert self.main_classifier_threshold is not None, "Main classifier's threshold must be set in the constructor"

        score = self.main_classifier.predict_proba(bytez)
        is_malicious = score > self.main_classifier_threshold
        if not is_malicious:
            score = self.EXE_scanner.get_score(bytez)
            is_malicious = score > self.EXE_scanner.threshold

        predicted_label = int(is_malicious)
        if return_score:
            return predicted_label, score
        else:
            return predicted_label

In [3]:
main_classifier = joblib.load("data/CLF_DT-dataset=EMBER.joblib")
EXE_scanner = EXEscanner()

wrapper = WrapperEXEscanner(main_classifier, EXE_scanner, main_classifier_threshold=0.5)

Name: EXE_scanner-GBDT-dataset=EMBER_benign_test-AE_all
Threshold: 0.9999941463527008
Num trees: 1000


In [4]:
SAMPLE_FOLDER = "samples"
for file in os.listdir(SAMPLE_FOLDER):
    with open(os.path.join(SAMPLE_FOLDER, file), "rb") as f:
        bytez = f.read()
        score = wrapper.get_score(bytez)
        print(f"{file}: {score}")